In [282]:
# **************************************************************************** #
#                                                                              #
#                                                         :::      ::::::::    #
#                                                       :+:      :+:    :+:    #
#                                                     +:+ +:+         +:+      #
#    By: luicasad <luicasad@student.42.fr>          +#+  +:+       +#+         #
#                                                 +#+#+#+#+#+   +#+            #
#    Created: 2023/10/22 11:17:16 by luicasad          #+#    #+#              #
#    Updated: 2023/10/22 11:17:24 by luicasad         ###   ########.fr        #
#                                                                              #
# **************************************************************************** #

In [283]:
import os
import pandas as pd
home_dir = os.getcwd()
data_folder =  home_dir +'/../../data/price/'

In [284]:
def concat_files(data_folder:str, years: range, common_name:str )-> pd.DataFrame:

    #For each year, 
    #   create a dataframe reading a csv file.
    #   Append it to a list
    #   Create a dataframe with columns count

    files = []
    counts =[]
    for year in years:
        file = data_folder + str(year) + common_name
        df = pd.read_csv(file)
        files.append(df)
        counts.append(pd.DataFrame(data=df.count(), columns=[year]))
    summary = pd.concat(counts, axis=1)
    difference = summary.iloc(0)[0]-summary.iloc(0)[-1]
    print(type(summary), type(difference))
    print(summary)
    print("-"*80)
    print("Any - Preu = NUM NaN")
    print(difference.to_frame().T)
    return pd.concat(files, ignore_index=True)

def count_NAN(df):
    """
    Count NAN values in each column of a dataset.
    Print 8 counts in per line
    """
    col_counter = 1
    for col in df.columns:
        count = df[col].isna().sum()
        if ((col_counter % 8) == 0):
            sep = "|\n"
        else:
            sep = "| "
        print("|{} = {}".format(col,count), end=sep)
        col_counter = col_counter + 1

<span style='color:red'> Llegir i concatenar fitxers de preus<br> </span><span style='color:green'> Lectura y concatenación de los archivos de precios<br></span><span style='color:blue'> Read and concatenate all prize files <br></span>

In [285]:
pd.set_option("display.width", 120)
price = concat_files(data_folder, range(2014, 2024),"_lloguer_preu_trim.csv")


<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
                2014  2015  2016  2017  2018  2019  2020  2021  2022  2023
Any              584   584   584   584   584   584   584   584   438   292
Trimestre        584   584   584   584   584   584   584   584   438   292
Codi_Districte   584   584   584   584   584   584   584   584   438   292
Nom_Districte    584   584   584   584   584   584   584   584   438   292
Codi_Barri       584   584   584   584   584   584   584   584   438   292
Nom_Barri        584   584   584   584   584   584   584   584   438   292
Lloguer_mitja    584   584   584   584   584   584   584   584   438   292
Preu             546   552   553   546   540   538   538   584   406   273
--------------------------------------------------------------------------------
Any - Preu = NUM NaN
   2014  2015  2016  2017  2018  2019  2020  2021  2022  2023
0    38    32    31    38    44    46    46     0    32    19


<span style='color:red'> Estudi dels valors Nan<br> </span><span style='color:green'> Estudio de los valores nulos<br></span><span style='color:blue'> Study Nan values <br></span>

In [286]:
count_NAN(price)

|Any = 0| |Trimestre = 0| |Codi_Districte = 0| |Nom_Districte = 0| |Codi_Barri = 0| |Nom_Barri = 0| |Lloguer_mitja = 0| |Preu = 326|


In [287]:
grouped_df = price[price["Preu"].isna()
                  ].groupby(['Any', 'Trimestre']).size().unstack()
pd.set_option('display.max_rows', None)
print("Trimestre    1      2      3      4   sum")
for row in grouped_df.iterrows():
    print("{}     ".format(row[0]), end=" ")
    suma = 0
    for data in row[1].values:
        print("{0:4}  ".format(data), end = " ")
        if (data != np.nan):
            suma += data
    print("{0:4}  ".format(suma), end = " ")
    print("{0:.2f}%  ".format(100*suma/584))



Trimestre    1      2      3      4   sum
2014      10.0    6.0   10.0   12.0   38.0   6.51%  
2015      12.0    4.0    8.0    8.0   32.0   5.48%  
2016       4.0   10.0    9.0    8.0   31.0   5.31%  
2017      10.0    8.0   10.0   10.0   38.0   6.51%  
2018      12.0   10.0   12.0   10.0   44.0   7.53%  
2019      12.0   12.0   12.0   10.0   46.0   7.88%  
2020       8.0   14.0   12.0   12.0   46.0   7.88%  
2022      12.0   10.0   10.0    nan    nan   nan%  
2023       9.0   10.0    nan    nan    nan   nan%  


In [288]:
Barris_with_NaN =price[price["Preu"].isna()]["Nom_Barri"].unique()
Capitalized = [s.capitalize() for s in Barris_with_NaN]

print("Barris amb preus NaN ==>", Capitalized)


Barris amb preus NaN ==> ['La marina del prat vermell', 'La clota', 'Torre baró', 'Vallbona', 'Baró de viver', 'Can peguera', 'Canyelles']


In [289]:
price[price["Preu"].isna()]["Codi_Barri"].unique()


array([12, 42, 54, 56, 58, 47, 49])

In [290]:
price["Codi_Barri"].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73])

In [291]:
price["Lloguer_mitja"].unique()

array(['Lloguer mitjà mensual (Euros/mes)',
       'Lloguer mitjà per superfície (Euros/m2 mes)'], dtype=object)

In [292]:
grouped_df = price[price["Preu"].isna()
                  ].groupby(['Codi_Barri', 'Lloguer_mitja']).size().unstack()
pd.set_option('display.max_rows', None)
print(grouped_df)

Lloguer_mitja  Lloguer mitjà mensual (Euros/mes)  Lloguer mitjà per superfície (Euros/m2 mes)
Codi_Barri                                                                                   
12                                            25                                           26
42                                            32                                           32
47                                            28                                           28
49                                             3                                            3
54                                            19                                           20
56                                            30                                           30
58                                            25                                           25


<span style='color:red'> Substitució dels valors Nan<br> </span><span style='color:green'> Substitución de los valores nulos<br></span><span style='color:blue'> Nan values <br></span>

In [293]:
# Only for Canyelles cause it is the barri with less missed ata

barri = 49
mask_all = (price['Lloguer_mitja'] == 'Lloguer mitjà per superfície (Euros/m2 mes)') & (price["Codi_Barri"] == barri)
price[mask_all]["Preu"] = price[mask_all]["Preu"].fillna(price[mask_all]["Preu"].rolling(3,min_periods=1).mean())
print(price[mask_all])

       Any  Trimestre  Codi_Districte Nom_Districte  Codi_Barri  Nom_Barri  \
340   2014          1               8    Nou Barris          49  Canyelles   
413   2014          2               8    Nou Barris          49  Canyelles   
486   2014          3               8    Nou Barris          49  Canyelles   
559   2014          4               8    Nou Barris          49  Canyelles   
924   2015          1               8    Nou Barris          49  Canyelles   
997   2015          2               8    Nou Barris          49  Canyelles   
1070  2015          3               8    Nou Barris          49  Canyelles   
1143  2015          4               8    Nou Barris          49  Canyelles   
1508  2016          1               8    Nou Barris          49  Canyelles   
1581  2016          2               8    Nou Barris          49  Canyelles   
1654  2016          3               8    Nou Barris          49  Canyelles   
1727  2016          4               8    Nou Barris          49 

/tmp/ipykernel_61299/3136143780.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price[mask_all]["Preu"] = price[mask_all]["Preu"].fillna(price[mask_all]["Preu"].rolling(3,min_periods=1).mean())


In [294]:
# Only for Canyelles cause it is the barri with less missed ata

barri = 49
mask_all = (price['Lloguer_mitja'] == 'Lloguer mitjà mensual (Euros/mes)') & (price["Codi_Barri"] == barri)
price["Preu"] = price[mask_all]["Preu"].fillna(price[mask_all]["Preu"].rolling(3,min_periods=1).mean())
print(price[mask_all])

       Any  Trimestre  Codi_Districte Nom_Districte  Codi_Barri  Nom_Barri                      Lloguer_mitja     Preu
48    2014          1               8    Nou Barris          49  Canyelles  Lloguer mitjà mensual (Euros/mes)   563.33
121   2014          2               8    Nou Barris          49  Canyelles  Lloguer mitjà mensual (Euros/mes)   474.29
194   2014          3               8    Nou Barris          49  Canyelles  Lloguer mitjà mensual (Euros/mes)   532.67
267   2014          4               8    Nou Barris          49  Canyelles  Lloguer mitjà mensual (Euros/mes)   566.36
632   2015          1               8    Nou Barris          49  Canyelles  Lloguer mitjà mensual (Euros/mes)  549.515
705   2015          2               8    Nou Barris          49  Canyelles  Lloguer mitjà mensual (Euros/mes)   575.83
778   2015          3               8    Nou Barris          49  Canyelles  Lloguer mitjà mensual (Euros/mes)    558.0
851   2015          4               8    Nou Bar